## <center><span style="color:yellow">Drugi domaci iz Digitanle obrade slike<span><center>
###  <span style="color:blue">Student       : Veljko Ilic<span>
    
###  <span style="color:blue">Br. indeksa   : 2017/0611<span> 
    
###  <span style="color:blue">Datum predaje : 05.12.2021<span>


#### Ova celija koda samo poziva sve potrebne biblioteke koje se koriste u izradi zadataka

In [ ]:
from pylab import *

import skimage
from skimage import color
from skimage import exposure
from skimage import filters
from skimage import util
from skimage import io
import imageio
from skimage import img_as_float

from scipy import ndimage
from scipy import signal as signal
import numpy as np

### <span style="color:yellow"><center>Filtriranje polutonirane slike<center><span>
Skripta koja se nalazi ispod ima pokaznu svrhu ako je potrebno da se neka druga RGB slika razbije na razvni i da se nadju Furijeove transforamcije tih ravni. Dakle ***SKRIPTA ISPOD NIJE OD VITALNOG ZNACJA ZA SAMO RESENJE*** jer ce slike koje sam prilozio ovde u izvetaju da se pojave i nakon pokretanja ove skripte. Ono sto vidimo na slikama spektara je da postoje odredjene frekvecnijske komponente na visokim ucestanostima jer za prirodne slike, a kakva Taj Mahal jeste nije van DC ima izrazene komponente. I ono sto ja pre svega zelim da uklonim jesu komponente na visokim frekvecnijama. Za to cu koristiti Gausov niskofrekventni filtar jer mije on dao najbolje rezultate.
    
![Medjurezutat](sekvence/planes_fft.jpg)


In [ ]:
#Unpackign half-toned image
img = img_as_float(imread('half_tone.jpg'));

#Because it's RGB image i will do fft2 on each plane. OFcourse spectre is centred
img_fft = np.zeros(img.shape,dtype=complex)
img_fft[:,:,0] = fftshift(fft2(img[:,:,0]));
img_fft[:,:,1] = fftshift(fft2(img[:,:,1]));
img_fft[:,:,2] = fftshift(fft2(img[:,:,2]));

#Ploting all three planes 
fig, axes = plt.subplots(nrows = 1,ncols = 3,figsize=[12,4], dpi=100);
ax = axes.ravel();

ax[0].imshow(img[:,:,0],cmap='jet');ax[0].set_axis_off(); ax[0].set_title('R ravan ulazne slike', fontsize=14);
ax[1].imshow(img[:,:,1],cmap='jet');ax[1].set_axis_off(); ax[1].set_title('G ravan ulazne slike', fontsize=14);
ax[2].imshow(img[:,:,2],cmap='jet');ax[2].set_axis_off(); ax[2].set_title('B ravan ulazne slike', fontsize=14);

#Ploting all three planes FFT 
fig, axes = plt.subplots(nrows = 1,ncols = 3,figsize=[12,4], dpi=100);
ax = axes.ravel();
ax[0].imshow(log(1+np.abs(img_fft[:,:,0])),cmap='gray');ax[0].set_axis_off(); ax[0].set_title('Magnituda spektra', fontsize=16);
ax[1].imshow(log(1+np.abs(img_fft[:,:,1])),cmap='gray');ax[1].set_axis_off(); ax[1].set_title('Magnituda spektra', fontsize=16);
ax[2].imshow(log(1+np.abs(img_fft[:,:,2])),cmap='gray');ax[2].set_axis_off(); ax[2].set_title('Magnituda spektra', fontsize=16);

#### Ovde sam kopirao kod koji generise *low-pass* filtar. Filtri koje je moguce generisati su ***Gaussian***,***Ideal***,***Baterworth***

In [ ]:
def lpfilter(filt_type, Ny, Nx, sigma, n=1):
    
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2

    
    X, Y = meshgrid(x, y)
    
    D = np.sqrt(np.square(X) + np.square(Y))
    
    if filt_type == 'gaussian':
        filter_mask = exp(-np.square(D)/(2*np.square(sigma)))
    elif filt_type == 'btw':
        filter_mask = 1/(1+(D/sigma)**(2*n))
    elif filt_type == 'ideal':
        filter_mask = ones([Ny, Nx])
        filter_mask[D>sigma] = 0
    else:
        print('Greška! Nije podržan tip filtra: ', filt_type)
        return
    
    return filter_mask

U celiji koda koja se nalazi ispod ove celije teksta je implementiran nacin da se sve tri ravni filtriranju low pass filtrom po zelji ja sam ostavio na tome da bude tipa ***gaussian***.

**Ideani filtar**
> Sto se tice idealnog fitra rezultat koji sam dobio nalazi se na slici ispod. Pre svega prelaz je jako ostar u idealnom fitru i kako god da ga menjam ja dobijam neka izoblicenja na drvecu kao da su "ustinuta" i da su se ivice umnozile.Slika je za ovaj filtar najmutnija.
![RGB,Idealni filtar](sekvence/rgb_ideal.jpg)

**Batervortov filtar**
>Sto se tice Batervortovog gde je red filtra 3 primecuje se da je slika i nije toliko mutna kao recimo za n=8 ali je se primecuju sitni detalji koji nastaju od toga sto nisu potisnute ove komponente koje su porstorperiodicne i to kvari utisak slike.
![RGB,Idealni filtar](sekvence/rgb_btw_n=3.jpg)
Za red fiultra osam nisam obuhvatio periodicne smetnje u spektru ali je slika postala mutnija.
![RGB,Idealni filtar](sekvence/rgb_btw_n=8.jpg)

**Gausov filtar**
>Kod Gausovog filtra najbolje je ispala slika gde je sigma=10. Dakle ja nigde nisam primeno NOC filtar da potisne ove smetnje koje se vide u spektru zeleo sam da prodjem sa manje rada. Te smetnje pokazu svoju posledicu na vrednosti sigma=18 recimo, gde se vec i vide tackice na slici tako da sam ostavio moj rezultat za sigma=10 kao finalnu.
![RGB,Idealni filtar](sekvence/rgb_gaussian_sigma=8.jpg)
![RGB,Idealni filtar](sekvence/rgb_gaussian_sigma=10.jpg)
![RGB,Idealni filtar](sekvence/rgb_gaussian_sigma=18.jpg)

In [ ]:
#Unpackign half-toned image
img = img_as_float(imread('half_tone.jpg'));

#Extracting image shape parameters
[Ny, Nx, layers] = shape(img)

#Creating new matrix with same shape as input image
img_filt = np.zeros(img.shape);

#Input image has 3 planes so each of them has to be proccesed
#Low-pass gaussin filter
filt_freq = lpfilter('gaussian', Ny, Nx, sigma=10);
img_fft_ = np.zeros(img.shape,dtype=complex)

for i in range(3):
    #Finding FFT of image and then centering that spectre
    img_fft = fftshift(fft2(img[:,:,i]))
    
    #Convolution is multiplying in freq. domain
    img_fft_filt = img_fft*filt_freq
    img_fft_[:,:,i] = img_fft_filt;
    
    #Inverse FFT
    img_filt[:,:,i] = real(ifft2(ifftshift(img_fft_filt)));
    img_filt[:,:,i] = np.clip(img_filt[:,:,i],0,1);

#Ploting final result compared to input half-toned image
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='jet'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_filt, cmap='jet'); ax[1].set_axis_off(); ax[1].set_title('Filtrirana slika');

### <span style="color:yellow"><center>Inverzno filtriranje slike ako je poznat kernel<center><span>
    
<span style='text-align:justify'>Pri izradi ove tacke domaceg zadatka pre svega sam prvo otpakovao sliku ulaznu koja je degradirana pomeranjem kamere pri fotografisanju. Koristeci ugradjenu funckiju ***fft2(...)*** nasao Furijeovu transforamciju degradirane slike.Dalje sam iskoristio ugradjenu funkciju koja mi centrira spektar ***fftshift(...)*** i to mi daje rezultat koji ce se koristiti pri filtriranju.Ispod su prikazani dobijeni rezultati.Samo da napomenem da su delovi koda koji plotutju ove medjurezultate zakomentarisani ali mozda neko bude zeleo da ubaci neku drugu sliku i zeli da prati sta se desava tokom filtriranja. O spektru mogu sa slike da zaljkucim da ima komponenti i na visokim ucestanostima ali o samom tipu koji je do promene doveo nista vise, zato nam i sluzi kernel koji smo dobili.<span> 

![Medjurezutat](sekvence/etf_and_etffft.jpg)
    
Slican postupak sam primenio i u nalazenju Furijeove transforamcije kernela. Stim da je kernel velicine 30x30, a kako znamo da se pri inverznom filtriranju zapravo izvrsava deljenje Furijeove transforamcije degradirane funkcije i Furijeove transformacije kernel(pokreta koji je IMU zabelezio, a koji je do degradacije i doveo) mi moramo da nekako da prosirimo jer je nama slika dimenzija $561x829$. Shodno tome ja sam u ugradjenoj funkciji ***fft2(...)***, pored kernela proseldio i parametar **s=img.shape**. Taj parametar ce mi omoguciti da se Furijeova transfomracija ulazne slike i kernela poklope po velicini. Kada se nadje FUrijeova transformacija kernela onda se spektar centrira. Na slici ispod su prikazani rezultati za nama zadat primer ali ce se oni kao sto vec napisao plotovati i pri pokretanju skripte.

![Medjurezutat](sekvence/kernel_fft.jpg)
    
Sto se tice igre sa parametrom jacine filtranja *k* ja sam mislio da cu ako budem njega podesavao moci da se resim ovih istnih belih tackica, ali zapravo mozda ulazna slika osim sto je degradacija nastal pomeranje kamere, mozda suma nije ni bilo. Jer koliko god da ja poemram ove tackice po meni ostaju gde su.
 
> Ako je k=1e-4    
 >> ![k=1e-4](sekvence/etf_blur_filt.jpg)    

> Ako je k=1e-10    
![k=1e-10](sekvence/etf_blur_filt.jpg)    

In [ ]:
#Importing blured image
img  = img_as_float(imread('etf_blur.tif'));

#Finding spectre of degraded image
#Image shape is 561x829 
img_fft = fft2(img);

#Centering that spectre
img_filt = fftshift(img_fft);

#Ploting centred spectre of input image
'''
fig, axes= plt.subplots(nrows=1, ncols=2,figsize=(12,4), dpi=120);
ax = axes.ravel()
ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika',fontsize=12);
ax[1].imshow(log(1+np.abs(img_filt)),cmap='gray'); ax[1].set_title('Magnituda centriranog spektra ulazne slike',fontsize=12);
ax[1].set_axis_off();
'''

#Importing kernel that will be used to filter the image
#Kernel shape is 30x30
kernel_imported = img_as_float(imread('kernel.tif'));

#Finding kernel fft, BUT parameter s{shape} in fft2() says length of each transformed axis of the output 
kernel_fft = fft2(kernel_imported,s=img.shape);

#Centering kernel_fft and then normalisation so that filter doesn't change DC value of image
kernel_filt = fftshift(kernel_fft)/np.sum(kernel_imported);

#Ploting imported kernel and kernel_fft
'''
fig1, axes1= plt.subplots(nrows=1, ncols=2,figsize=(12,4), dpi=100)
ax1 = axes1.ravel();
ax1[0].imshow(kernel_imported, cmap='gray'); ax1[0].set_title('Ulazna slika kernela',fontsize=12); ax1[0].set_axis_off();
ax1[1].imshow(log(1+np.abs(kernel_filt)),cmap='gray');  ax1[1].set_title('Magnituda centriranog spektra kernela',fontsize=12);
ax1[1].set_axis_off();
'''
#Wiener filter parameters
k = 1e-4
W = (np.abs(kernel_fft)**2)/(np.abs(kernel_fft)**2 + k);

#Filering input image, and the limiting output result via Wiener filter
img_filt_est = (img_filt / kernel_filt)*W;
#Inverse fft2 of filtered image
img_est = real(ifft2(ifftshift(img_filt_est)));
#Clipin output result of ifft2 so it fits in range 0 to 1
img_est = np.clip(img_est,0,1);

#Ploting final result
fig2, axes2= plt.subplots(nrows=1, ncols=2,figsize=[12,4], dpi=120)
ax2 = axes2.ravel();
            
ax2[0].imshow(img,cmap='gray'); ax2[0].set_axis_off();ax2[0].set_title('Degradirana slika',fontsize=12);
ax2[1].imshow(img_est,cmap='gray');ax2[1].set_title('Filtrirana slika',fontsize=12); ax2[1].set_axis_off();

### <span style="color:yellow"><center>Restauracija slike koja ima nepoznatu kolicinu Gausovog suma<center><span>
    
Pre svega sam otpakovao ulaznu sliku *lena_noise.tif* i zatim definisao parametar *r* pomocu kod cu menjati velicinu okruzenja oko piksela. Kako je velicina prozora neparan broj otud i formula *(2xr+1)x(2xr+1)*. Nakon toga sleduje deo koda gde je potrebno naci lokalne varijanse. Postoji i kraca formula koja kaze da je varijansa jednaka razlici srednje vrednosti kvadrata i kvadrata srednje vrednosti i to sad treba da se primeni na lokalno susedstvo. Dakle tu koristimo ovaj *box filtar* koji se sastoji samo od jedinica, a koji je i naravno noramalizovan. Da se ne bi pisale petlje moze se iskoristiti ugradjena funkcija za korelaciju koja ce sada sa zadatim filterom proci kroz prosledejenu joj sliku i naci sumu svih piksela u okrizenju za svaki piksel slike.Da bi se nasao kvadrat srednje vrednosti primeni se slicno razmisaljnje stim da se ulazna slika pre toga kvadrira(tj. da se svaki njeg piksela kvadrira). Sada smo nasali lokalne varijanse i potrebno je sa histogram odrediti nepoznati varijansu suma.
    
Da bih nasao histogram ja sam iskoristi ugradjenu funkciju iz *numpy* biblioteke, koja nalazi ponavaljanja svih elemenata u nizu. Kao povratnu vrednsot vraca koji su elementi u nizu jedinstveni i vraca koliko se puta oni ponavaljaju u ulaznom nizu odnosno matrici. Sada plan za nalazenje varijanse suma slike je da nadjem na histogram onaj bin koji ima najveci pik i ako postoji vise takvih pikova koji su maksimalni onda izaberem medijan binova koji daju najvise pikove.Zasto sam to usvojio na taj nacin? Ja sam razmisljanje krenuo od prvog histograma koji sam dobio za vrednost $r=3$. Vidi se da se vrednosti rasipaju od centralgno pika ka krajevima, ako bih uzeo neki varijansu koja je dosta velika onda ne bih mogao da sacuvam ivicu, a ako uzmem neku vrednost levo od pika koja je dosta manja onda bih dobio sliku slicnu ulaznoj. Dalje sam razmislja sta ako za varijansu suma onu vrednost koja se najvise pojavljue, tj pik. Jer ako po postavci zadatka uzmem da su skoro svi regioni uniformni onda bi i varijanse trebale da im budu priblizno iste. I zato uzimam varijanse koje se najvise pojavaljuju, jer se nadam da cu onda imati najvecu osetljivost kada sam naisao na ivicu.

E sad ako pogledamo histograme koje sam ispod prikazao vidimo da se sto je veci prozor vrednosti na histogramu grupisu tj kao da raspodela postaje uniformna. Kako je prozor veci dobijamo da se sve manje i manje regiona smatra uniformnim i samim tim filtranje gubi na kvalitetu 
>![r=3](sekvence/final3_r=3.jpg)
    >>![histogram1](sekvence/hist_r=3.jpg)
    
>![r=7](sekvence/final3_r=7.jpg)
    >>![hoistogram2](sekvence/hist_r=7.jpg)

>![r=12](sekvence/final3_r=12.jpg)
    >>![hits](sekvence/hist_r=12.jpg)

>![r=18](sekvence/final3_r=18.jpg)
    >>![hist4](sekvence/hist_r=18.jpg)

In [ ]:
#Importing input image with noise
img = img_as_float(imread('lena_noise.tif'));

#Parameter radius is changable "by hand" 
radius = 3;
#Size of window is odd number, there for size is equal 2*r+1
size = 2*radius+1

#Filter shape (2*r+1)x(2*r+1), it constists of ones and is normalizes with size squared
filt_avg = ones([size,size])/(size**2);

#Finding local average value for each pixel's neighbourhood
img_local_avg = ndimage.correlate(img, filt_avg);

#Finding square local averade value for each pixel's  neighbourhood
img_sqr_local_avg = ndimage.correlate(img**2, filt_avg);

#Calculating local varinace via this formula below
img_local_var = img_sqr_local_avg - img_local_avg**2;


#Finding parameters for histogram, by counting unique elements frequency
(unique, counts) = np.unique(img_local_var, return_counts=True);

#Estimating noise varinace from histogram as the bin with the highest peak, if there is multiple highest peaks then
#median is calculated
peak = np.max(counts);
peak_index = np.where(counts==np.max(counts))
noise_var = np.median(unique[peak_index]);
figure;
plt.plot(unique,counts);plt.title("Histogram lokalnih varijasni za r=3")
weight = noise_var/img_local_var
weight[weight>1]=1

img_est = img + weight*(img_local_avg-img);

fig, axes = plt.subplots(nrows = 1,ncols = 3,figsize=[12,4], dpi=120);
ax = axes.ravel()
ax[0].imshow(img,cmap='gray');ax[0].set_title('Ulazna slika pogodjena sumom',fontsize=12);
ax[1].imshow(weight,cmap='gray');ax[1].set_title('Slika tezinskog vektora',fontsize=12);
ax[2].imshow(img_est,cmap='gray');ax[2].set_title('Filtrirana slika',fontsize=12);

### <span style="color:yellow"><center> Implementacija funckije koja vrsi filtriranje impulsnog suma i obicni i adaptivnim metodom<center><span>
    
Pri implementaciji ove funkcije prvo sto sam uradio jeste proverio da li su podaci koji joj se prosledjuju validni ili ne. Evo kako sam ja to shvatio iz teksta koji nam je dat. Ukoliko je parametar *s_max* manji od nule funkcija vraca gresku, ukoliko nije ceo broj onda ce funkcija da kastuje broj u tip *int* i o tome ce obavestiti korisnika, zatim se proverava da li je broj neparan ili ne.Ako je neparan nastavlja se sa izvrsavanje dalje. Dalje se kreira parametar *img_out* koji ce biti tre
nutna promenljiva u koju se smestaju obradjeni pikseli. Dakle ona je istih dimenzija kao i ulazna slika.
    
Dalje se proverava da li korisnik zeli da se sprovede obicni ili adaptivni medijan filtar i shodno tome se kod dalje izvrsava.
    
Obicni medijan filtar se izvrsava tako sto se pomocu dve *for* petlje prodje kroz ulaznu sliku i za svaki piksel se poziva funkcija koju sam nazvao ***get_median***. Funkcija ***get_median*** ce iracunati medijan lokalnog susedstva tako sto ce pre svega da izracuna da li prozor koji oznacava okolinu iz koje se racuna izlazi van okvira same slike.Primenio sam jednostavnu formulu da nadjem koordinate coskova prozora iz kog se racuna medijan. Naime ako je neki piskel sa koordinatam $(x,y)$,a ja znam da prozor mora imati velicinu koja je neparna onda mi je sredisnji piksel podjednako udaljen od stranica prozora.Udaljenost od leve ivica prozora  $y-(s_{max}-1)/2$. Ukoliko neka od ovih vrednosti je veca od visine, sirine ili manj od nule onda se ona zakucava na tu vrednost.Te 4 koordinate su sada zapravo granice za moje dve *for* petlje koje ce proci kroz ulaznu sliku ali samo izmedju ovih koordinata i odatle prikupitiinforamcije o intezitetima piksela u okruznjeu.

Sledeci na redu je algoritam koji sortira u niz prikupljene informacije. Iz tako sortiranog niza se dalje trazi medijan. Pronadjeni medijan i sortirani niz se vracju kao povratne vrednosti. Medijan se upisuje na koordinate koje odgovaraju koordinata ulaznog piksela u ulaznoj slici.Tako obradjena slika se vraca nazad korisniku.

>Slika koju sam ja odabra:
    >>![Pablo](sekvence/pablo.jpg)

Evo nekoliko primera za obicni medijan filtar.    
> amount = 0.2 window_size = 3x3
    >>![amount0.2 3x3](sekvence/amount0.2_3x3.jpg)
    
> amount = 0.2 window_size = 5x5
   >> ![amount0.2 5x5](sekvence/amount0.2_5x5.jpg)

> amount = 0.4 winow_size = 7x7
  >>![amount0.4 7x7](sekvence/amount0.4_7x7.jpg)

> amount = 0.6 window_size=9x9
  >> ![amount0.6 9x9](sekvence/amount0.6_9x9.png)   

> amount = 0.8 window_size=11x11    
   >> ![amount0.8 11x11](sekvence/amount0.8_11x11.png)
 
> I na kraju brzina funkcije    
   >> ![speed](sekvence/brzina.jpg)

Sto se tice komentara na slike manje vise rade filtri isto jedino me buni kako sam ja za 9x9 dobio bolju sliku. Slike su naravno sto je prozor odakle se trazi medijan vecin mutnije, jer mi pre svega ovde radimo usrednjavanje.
    
Sto se tice adaptivnog filtriranja opet zavrtim dve for petlje ali sad za svaki piksel se prvo postavi pocetna vrednost minimalne vrednosti prozora na 3. Zatim se proverava da li je potrebno povecavati prozor. Treba da kazem isto i da ja sada opt koristim funkciju ***get_median***, a ciji sam nacin funkcionisanja opisao malopre.
    
    
Evo nekih primera i za adaptivni medijan:
> amount=0.4 max_window_size=7x7  izvrsi se za nekih $24,6[s]$ sto je dosta bolje od 4 minuta za obicni filtar.
   >> ![adap_median](sekvence/adap_median0.4_7x7.jpg)

> amount=0.6 max_window_size=9x9  izvrsi se za nekih $51[s]$ sto je mnogo bolje od 10 minuta za obicni filtar.
    Mozda jeste malo lose to sto imamo ove crne negde bele tackice ali su povecati maks vrednsot prozora.
   >> ![adap_median1](sekvence/adap_median0.6_9x9.jpg)
    
> amount=0.6 max_window_size=11x11  izvrsi se za nekih $51[s]$ sto je mnogo mnogo bolje od 10 i kusur minuta za obicni filtar.
   >> ![adap_median2](sekvence/adap_median0.6_11x11.png)
    
    
Molim Vas da ako uocite sta sam pogresio u ovom adaptivno da mi posaljete neki feedback, jer ova vremena deluju basnoslovno, i nekako ne mogu da poverujem da i sa velikom koncetracijom suma od 0.6 dobijem ovakvu izlaznu sliku.

In [ ]:
def get_median(img_in,s_max,x,y):
    
    #Finding box edge coordinates, where i look for median
    s_half = (s_max-1)//2
    xu = x-s_half
    xd = x+s_half
    yl = y-s_half
    yr = y+s_half
    
    #What in case of edge pixels? Okruzenje mora biti prekrojeno.
    if(xu < 0):
        xu = 0;
    if(xd >= img_in.shape[0]):
        xd = img_in.shape[0]-1;
    if(yl < 0):
        yl = 0;
    if(yr >= img_in.shape[1]):
        yr = img_in.shape[1]-1;
    
    box_flatten = np.zeros((xd-xu+1)*(yr-yl+1),dtype=type(img_in[0][0])); 
    
    k = 0;
    for i in range(xu,xd+1):
        for j in range(yl,yr+1):
            box_flatten[k] = img_in[i][j];
            k+=1;
            
    for i in range(0,box_flatten.size -1):
        for j in range(i+1,box_flatten.size):
            if(box_flatten[i] > box_flatten[j]):
                t = box_flatten[j];
                box_flatten[j] = box_flatten[i];
                box_flatten[i] =t;
                                         
    #Finding median
    if(box_flatten.size//2 != 0):
        median = box_flatten[(box_flatten.size+1)//2];
    else:
        median = (box_flatten[box_flatten.size//2]+box_flatten[box_flatten.size//2-1])/2;
        
        median = floor(median);
            
    return median, box_flatten;

def dos_median(img_in, s_max, adaptive=False):
    '''
    "Median and Adaptive median filter implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Funkcija dos_median filtrira ulaznu sliku medijan filtrom. Postoje dva nacina da se to uradi obicnim 
    medijan filtrom gde se svaki piksel ulazne slike menja sa medijanom njegove okolina. Okolina ili ti 
    prozor se zadaje preko parametra s_max i mora biti neparan broj. Da li ce se izvrsiti adaptivni medi-
    jan filtar ili obicni medijan odredjuje parametar adaptive koji je p difoltu postavljen na False 
    vrednost. Ako je adaptive=True onda funkcija sprovodi apaptivni medijan filtar.
    
    Inputs
    
      imgIn    - Ulazna slika koju treba obraditi i ona mora biit ili u unit8 ili float64 formatu
      s_max    - Parametar preko kog se zadaje velicina prozora
      adaptive - Sluzi da funkcija napravi razliku da li da sporvede adaptivni ili obicni medijan 
      filtar.
    Outputs
    
      img_out - Obradjena slika istih dimenzija kao i ulazna
    
    Example
    img_filtered = dos_median(img_in,5)
    img_filtered = dos_median(img_in,5,True)
    '''
    #Check if correct digit is entered for maximum size window 
    if(s_max <= 0):
        print('Maximum size of window has to be possitive nuber!');
        return False;
    elif(type(s_max) != int):
        print('Maximum size of window has to be integer, program will convert it into type INT and continue executing!');
        s_max = int(s_max)+1;
        print('new_s_max = ', s_max);
    
    #Check if max. window size is odd number
    if(s_max % 2 == 0 or s_max == 1):
        print('s_max has to be odd and greater than one!');
        return False; 
    
    #Function accepts two type of data format uin8 and double.Code block below check that condition
    if(type(img_in[0][0]) != uint8 and type(img_in[0][0]) != double):
        print('Data type of input image must be uint8 or double!');
        return False;
    
    #Creating output image
    img_out = np.zeros(img_in.shape,dtype=type(img_in[0][0]));
    #Izvalcenje podataka o visini i sirini slike
    [height, width] = img_in.shape;
    
    if(adaptive):
        for i in range(height):
            for j in range(width):
                s_min = 3;
                
                median, box_flatten = get_median(img_in,s_min,i,j);
                while(True):
                    
                    if((s_min==s_max) or ((median!=box_flatten[0]) and (median!=box_flatten[box_flatten.size-1])) ):
                        break;
                    else:
                        s_min+=2;
                    
                    median, box_flatten = get_median(img_in,s_min,i,j);
                    
                #If current window size is already max then output pixel value is median.
                if(s_min==s_max):
                    img_out[i][j] = median;  
                elif(box_flatten[0]==img[i][j] or (box_flatten[box_flatten.size-1]==img[i][j])):
                    img_out[i][j] = median;
                else:
                    img_out[i][j] = img[i][j];
                    
        
    else:
        for i in range(height):
            for j in range(width):
                img_out[i][j], _ = get_median(img_in,s_max,i,j);
    return img_out

In [ ]:
import time

img = imread('sekvence/pablo.jpg');
img_impulse_noise = util.random_noise(img, mode='s&p', amount=0.6)

start = time.time()
img_noise_clear =  dos_median(img_impulse_noise,11,True);
end = time.time();
execution_time = end-start;

print("Vreme izvrsavanja: " + str(round(execution_time,3))+ "s \n")
execution_time_norm = execution_time/np.size(img)
print("Vreme izvrsavanja: " + str(round(execution_time_norm*1e6,7))+ " us/pix \n")

#img_filt_signal = signal.medfilt(img_impulse_noise, kernel_size=3);
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=[10,4], dpi=100);
ax[0].imshow(img,cmap='gray');ax[0].set_title('Originalna slika',fontsize=12);
ax[1].imshow(img_impulse_noise, cmap='gray');ax[1].set_title('Slika sa sumom',fontsize=12);
ax[2].imshow(img_noise_clear, cmap='gray');ax[2].set_title('Adaptivni medijan filtar',fontsize=12);

In [ ]:
window_size = [9,25,49,81,121];
speed = [51.159,256.716,853.007,2057.144,4164.807];
figure, ax = plt.subplots(figsize=[12,6])
plt.plot(window_size,speed);plt.title("Brzina funkcije dos_median");
plt.xlabel("Velicina prozora $[pixels]$");
plt.ylabel("Normalizovano vreme $[us/pix]$");

<center>Tabela vremena u zavistnosi od velicine prozora za obicni medijan filtar <center>

|Kolicina suma      |Velicina prozora   |Vreme izvrsavanja [s] |Noramlizovnao vreme [us/pix]|
|-------------------|-------------------|----------------------|----------------------------|
|      0.2          |        3x3        |         15,65        |         51,159             |
|      0.2          |        5x5        |         78,55        |        256,716             |
|      0.4          |        7x7        |        261,02        |        853,007             |
|      0.6          |        9x9        |        629.486       |       2057,144             |
|      0.8          |       11x11       |       1274,431       |       4164,807             |